In [ ]:
using Plots
using JuMP
using MadNLP
using ProgressMeter

Classical Lennard-Jones potential (see also https://en.wikipedia.org/wiki/Lennard-Jones_potential):
$$
    W_{x, x'}(r) := 4 \left[
        \left(\frac{r}{|x - x'|}\right)^{12}
        - \left(\frac{r}{|x - x'|}\right)^6
    \right].
$$

In [ ]:
# Lennard-jones of squared distances!
function lennard_jones(cur_dist_sq; pref_dist_sq=1, pot_min=-1)
    # the factor 2^(1/6) assures that the global
    # minimum is attained at cur_dist == pref_dist
    q = pref_dist_sq/(2^(1/3)*cur_dist_sq)
    -4*pot_min*(
        q^6 - q^3
    )
end

In [ ]:
pot_min = -2
pref_dist = 1.5
xs = range(pref_dist/2, 5, length=1000)
f(r) = lennard_jones(r, pref_dist_sq=pref_dist^2, pot_min=pot_min)
ys = f.(xs.^2)
plot(xs, ys, ylims=(pot_min-0.5, 7), label="W")

In [ ]:
N = 4
max_stretch = .5
time_horizon = 1.
fps = 30
pref_dist = 1/(N-1)
pot_min = -1.
elastic_thresh = 2*pref_dist
search_rad = 1.
diss_coeff = 1.
soft_max_alpha = 1e1
eps = 0.5*pref_dist

In [ ]:
function dirichlet_bdry_cond(t; max_stretch=1, time_horizon=1)
    max_stretch*sin(2*pi/time_horizon*t)
end
g(t) = 1 + dirichlet_bdry_cond(t; max_stretch=max_stretch, time_horizon=time_horizon)
xs = range(0, 1, length=1000)
ys = g.(xs)
plot(xs, ys, label="g", ylims=(0, 1 + max_stretch))

In [ ]:
soft_max(x, y) = (x*exp(soft_max_alpha*x) + y*exp(soft_max_alpha*y)) / 
    (exp(soft_max_alpha*x) + exp(soft_max_alpha*y))
xs = range(-2., 2., length=1000)
ys = [soft_max(x, 0.) for x in xs]
plot(xs, ys)

In [ ]:
step = 1
minmove = Model(
    optimizer_with_attributes(
        MadNLP.Optimizer,
#         "print_level" => MadNLP.WARN,
        "blas_num_threads" => 4
    )
)

@NLparameter(minmove, prev_y[i=1:N] == (i-1)*pref_dist)
@NLparameter(minmove, max_elong_sq[i=1:N-1] == pref_dist^2)

@variable(
    minmove, 
    value(prev_y[i]) - search_rad <= y[i=1:N] <= value(prev_y[i]) + search_rad
)

# Dirichlet condition on the right endpoint
@constraint(
    minmove, dirichlet,
    y[N] == g(step*time_horizon/fps)
)
# Regularization to avoid large values
@expression(minmove, dist_sq[i=1:N-1], (y[i+1] - y[i])^2)
@constraint(
    minmove, dist_sq_nonzero[i=1:N-1],
    dist_sq[i] >= eps^2
)

register(minmove, :soft_max, 2, soft_max; autodiff = true)
# @NLexpression(
#     minmove, damaged_dist_sq[i=1:N-1],
#     soft_max((y[i+1] - y[i])^2, max_elong_sq[i])
# )
@NLexpression(minmove, damaged_dist_sq[i=1:N-1], dist_sq[i])
W(dist_sq) = lennard_jones(dist_sq, pref_dist_sq=pref_dist^2, pot_min=pot_min)
register(minmove, :W, 1, W, autodiff = true)
@NLexpression(
    minmove, energy, 
    .5*sum(W(damaged_dist_sq[i]) for i in 1:N-1)
)
@NLexpression(
    minmove, dissipation,
    .5*sum((y[i] - prev_y[i])^2 for i in 1:N)
)
@NLobjective(minmove, Min, energy + diss_coeff*fps*dissipation)

minmove

In [ ]:
optimize!(minmove)

In [ ]:
xs = range(0, 1, length=N)
ys = value.(y)
plot(xs, ys)

In [ ]:
g(step*time_horizon/fps)

In [ ]:
value.(ys)

In [ ]:
value(dissipation)

In [ ]:
value(energy)